In [273]:
import pandas as pd

In [274]:
df=pd.read_excel(r"C:\Users\SRIKANTH\Desktop\OpenCTI_Data.xlsx",sheet_name='intrusion-set')

In [275]:
df.keys()

Index(['internal_id_key', 'stix_id_key', 'stix_label', 'entity_type',
       'created', 'modified', 'revoked', 'object_status', 'created_at',
       'created_at_day', 'created_at_month', 'created_at_year', 'updated_at',
       'name', 'description', 'alias', 'graph_data', 'first_seen',
       'first_seen_day', 'first_seen_month', 'first_seen_year', 'last_seen',
       'last_seen_day', 'last_seen_month', 'last_seen_year', 'goal',
       'sophistication', 'resource_level', 'primary_motivation',
       'secondary_motivation', 'type', 'id', 'labels', 'external_references',
       'object_marking_refs', 'granular_markings'],
      dtype='object')

In [233]:
# data=df['intrusion-set'][4:]
# columns=list(df['intrusion-set'].iloc[3])
# data.columns=columns
# data.reset_index(inplace=True)
# data.drop('index',axis=1,inplace=True)

In [276]:
import uuid
from datetime import datetime
import numpy as np

In [397]:
def  insert_general_info(df,ent_type,create):
    if create:
        for index,i in df.iterrows():
            df.internal_id_key[index]=str(uuid.uuid4())
            df.stix_id_key[index]=ent_type+'--'+str(uuid.uuid4())
            df.stix_label[index]='NA'
            df.entity_type[index]=ent_type
            df.created=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at_day=(datetime.now()).strftime("%Y-%m-%d")
            df.created_at_month=(datetime.now()).strftime("%Y-%m")
            df.created_at_year=(datetime.now()).strftime("%Y")
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.revoked=False
            df.object_status='NA'
    else:
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
    return df
    

In [398]:
result=insert_general_info(df=df,ent_type='Intrusion-set',create=True)
result.fillna('NA',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is t

In [279]:
from grakn.client import GraknClient
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [411]:
def insert_one_intrusion_set(df,session):
    print(df['name'])
    print(df['first_seen_day'])
    #print(count)
    if df.first_seen != 'NA':
        f=df.first_seen.strftime("%Y-%m-%dT%H:%M:%S")
        f_m=df.first_seen.strftime("%Y-%m")
        f_d=df.first_seen.strftime("%Y-%m-%d")
    elif df.first_seen_month != 'NA':
        f_m=df.first_seen_month
    elif df.first_seen_day != 'NA':
        f_d=df.first_seen_day
    else:
        f_d=None
        f_m=None
        f=None
               
    aliases=df.alias.split('\n')

    graql_match_query = f"match $i isa Intrusion-Set, " \
                        f"has name \"{df['name']}_\"; " \
                        f"get;"
    print(graql_match_query)
    with session.transaction().read() as read_transaction:
        # read_transaction.query(graql_match_query)
        answer_iterator = None
        answer_iterator = read_transaction.query(graql_match_query, infer=True)
        data = answer_iterator.collect_concepts()
                
        if len(data) > 0:
            # Update data.
            print("Update intrusion-set...")
        else:
            print("Insert intrusion-set...")
            graql_insert_query = f"insert $x isa Intrusion-Set, " \
                                 f"has internal_id_key \"{df['internal_id_key']}\", " \
                                 f"has stix_id_key \"{df['stix_id_key']}\", " \
                                 f"has stix_label \"{df['stix_label']}\", " \
                                 f"has entity_type \"{df['entity_type']}\", " \
                                 f"has created {df['created']}, " \
                                 f"has revoked \"{df['revoked']}\", "\
                                 f"has created_at {df['created_at']}, " \
                                 f"has created_at_day \"{df['created_at_day']}\", " \
                                 f"has created_at_month \"{df['created_at_month']}\", " \
                                 f"has created_at_year \"{df['created_at_year']}\", "\
                                 f"has updated_at {df['updated_at']}, "\
                                 f"has name \"{df['name']}_\" "
                                 
            if df['description'] != 'NA':
                graql_insert_query += f", has description \"{df['description']}\" "
            if len(aliases) > 0:
                s = "|"
                graql_insert_query += f", has alias \"{s.join(aliases)}\" "
            if f is not None:
                graql_insert_query += f", has first_seen {f} "
            if f_d is not None:
                graql_insert_query += f", has first_seen_day \"{f_d}\" "
            if f_m is not None:
                graql_insert_query += f", has first_seen_month \"{f_m}\" "
            if f_m is not None:
                graql_insert_query += f", has first_seen_year \"{df['first_seen_year']}\""
            graql_insert_query += ";"
            print(graql_insert_query)
            with session.transaction().write() as write_transaction:
                # make a write transection with the query
                write_transaction.query(graql_insert_query)
                # remember to commit at the end
                write_transaction.commit()

In [285]:
def load_data_into_grakn(session,input_df):
    print("Inserting intrusion-set...")
    # using progress_apply instead of apply so we have a progress bar form tqdm
    # input_df.progress_apply(insert_one_intrusion_set, axis=1, session=session)
    for index,row in input_df.iterrows():
        insert_one_intrusion_set(df=row,session=session)

In [282]:
def build_grakn_graph(input_df, keyspace_name):
    with GraknClient(uri="192.168.83.128:48555") as client:
        with client.session(keyspace = keyspace_name) as session:
            load_data_into_grakn(session,input_df)

In [412]:
build_grakn_graph(input_df=result,keyspace_name='grakn')

Inserting intrusion...
APT12
2012-10-01 00:00:00
match $i isa Intrusion-Set, has name "APT12_"; get;
1
Update intrusion-set...
APT28
NA
match $i isa Intrusion-Set, has name "APT28_"; get;
1
Update intrusion-set...
BlackOasis
06-5-2015

match $i isa Intrusion-Set, has name "BlackOasis_"; get;
1
Update intrusion-set...
Cobalt
2016-01-06 00:00:00
match $i isa Intrusion-Set, has name "Cobalt_"; get;
1
Update intrusion-set...
APT30
2004-08-16 00:00:00
match $i isa Intrusion-Set, has name "APT30_"; get;
1
Update intrusion-set...
APT12
2019-12-28 00:00:00
match $i isa Intrusion-Set, has name "APT12_"; get;
1
Update intrusion-set...


In [244]:
result['first_seen']

0   2012-10-01 00:00:00
1                   NaT
2   2015-05-06 00:00:00
3   2016-01-06 00:00:00
4   2004-08-15 00:00:00
5   2019-12-28 06:13:42
Name: first_seen, dtype: datetime64[ns]

In [50]:
from grakn.client import GraknClient

with GraknClient(uri="192.168.83.128:48555") as client:
    with client.session(keyspace="grakn") as session:
        ## Insert a Person using a WRITE transaction
        with session.transaction().write() as write_transaction:
            insert_iterator = write_transaction.query('insert $x isa Intrusion-Set,has internal_id_key "212ca911-1936-417e-a6bf-a834569ea8dq", has stix_id_key "Intrusion-set--76851f60-9948-49f0-874b-daec774999b5", has stix_label "NA", has entity_type "Intrusion-set",has created'+datetime.now()+';')
            ## to persist changes, write transaction must always be committed (closed)
            write_transaction.commit()

TypeError: can only concatenate str (not "datetime.datetime") to str

In [207]:
result['first_seen']

0    2012-10-01 00:00:00
1                     NA
2            06-5-2015\n
3    2016-01-06 00:00:00
4    2004-08-15 00:00:00
5    2019-12-28T06:13:42
Name: first_seen, dtype: object

In [53]:
import datetime
date = datetime.datetime(year=2018, month=8, day=6) # requires `import datetime`
birth_date = birth_date_type.create(date)

NameError: name 'birth_date_type' is not defined

In [54]:
date

datetime.datetime(2018, 8, 6, 0, 0)

In [170]:
with GraknClient(uri="192.168.83.128:48555") as client:
    with client.session(keyspace="grakn") as session:
        with session.transaction().read() as read_transaction:
            answer_iterator = read_transaction.query('match $i isa Intrusion-Set, has entity_type$i-m,has internal_id_key $i-x,has name "APT12"; get;  offset 0; limit 30;')
            persons = answer_iterator.collect_concepts()
            #read_transaction.commit()

In [171]:
persons

In [169]:
for person in persons:
                print("Retrieved person with id "+ person)

TypeError: can only concatenate str (not "Attribute") to str

In [37]:
build_grakn_graph(input_df=result,keyspace_name='grakn')

Inserting intrusion...



  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

0
0


GraknError: ('Server/network error: <_Rendezvous of RPC that terminated with:\n\tstatus = StatusCode.UNKNOWN\n\tdetails = "syntax error at line 1: \nmatch $x isa Intrusion-Set, has name "APT12";insert $x isa Intrusion-Set,has internal_id_key "212ca911-1936-417e-a6bf-a834569ea8db", has stix_id_key "Intrusion-set--76851f68-9948-49f0-874b-daec774999b5", has stix_label "NA", has entity_type "Intrusion-set", has created 2020-01-04T10:12:53, has revoked False,has object_status "NA", has created_at "2020-01-04T10:12:53", has created_at_day "2020-01-04", has created_at_month 2020-01, has created_at_year 2020;has updated_at 2020-01-04T10:12:53;has name "APT12_", has description "APT12\xa0is a threat group that has been attributed to China. The group has targeted a variety of victims including but not limited to media outlets, high-tech companies, and multiple governments", has alias "[\'APT 12 \', \'Numbered Panda \', \'TG-2754 \', \'BeeBus \', \'Calc Team\', \'Group 22 \', \'Crimson Iron \']",

In [173]:
for index,i in result.iterrows():
    print(i.stix_label)

NA
NA
NA
NA
NA
NA


In [98]:
result.name

0         APT12
1         APT28
2    BlackOasis
3        Cobalt
4         APT30
5         APT12
Name: name, dtype: object

In [119]:
for index,i in result.iterrows():
    print(i)

NaN                                                                    NA
internal_id_key                      212ca911-1936-417e-a6bf-a834569ea8db
stix_id_key             Intrusion-set--76851f68-9948-49f0-874b-daec774...
stix_label                                                             NA
entity_type                                                 Intrusion-set
created                                               2020-01-04T10:12:53
modified                                              2020-01-04T10:12:53
revoked                                                             False
object_status                                                          NA
created_at                                            2020-01-04T10:12:53
created_at_day                                                 2020-01-04
created_at_month                                                  2020-01
created_at_year                                                      2020
updated_at                            

In [32]:
data['name']

4         APT12
5         APT28
6    BlackOasis
7        Cobalt
8         APT30
9         APT12
Name: name, dtype: object

In [11]:
data['description']

4    APT12 is a threat group that has been attribut...
5    APT28 is a threat group that has been attribut...
6    BlackOasis is a Middle Eastern threat group th...
7    Cobalt Group is a financially motivated threat...
8    APT30 is a threat group suspected to be associ...
Name: description, dtype: object

In [12]:
data['first_seen']

4    2012-10-01 00:00:00
5                    NaN
6            06-5-2015\n
7    2016-01-06 00:00:00
8    2004-08-15 00:00:00
Name: first_seen, dtype: object

In [13]:
data

,NaN,internal_id_key,stix_id_key,stix_label,entity_type,created,modified,revoked,object_status,created_at,...,sophistication,resource_level,primary_motivation,secondary_motivation,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
data.columns

Index([                   nan,      'internal_id_key',          'stix_id_key',
                 'stix_label',          'entity_type',              'created',
                   'modified',              'revoked',        'object_status',
                 'created_at',       'created_at_day',     'created_at_month',
            'created_at_year',           'updated_at',                 'name',
                'description',              'aliases',           'graph_data',
                 'first_seen',       'first_seen_day',     'first_seen_month',
            'first_seen_year',            'last_seen',        'last_seen_day',
            'last_seen_month',       'last_seen_year',                'goals',
             'sophistication',       'resource_level',   'primary_motivation',
       'secondary_motivation',                    nan,                    nan,
                          nan,                    nan,                    nan,
                          nan],
      dtype='object'

In [15]:
import grakn

In [17]:
from grakn.client import GraknClient
import pandas as pd
from tqdm import tqdm
#tqdm.pandas()